In [13]:
import pandas as pd
from datetime import timedelta

In [4]:
df_imu = pd.read_csv(
    "D:/thesis_code/datasets/monado_slam/MOO07/MOO07_mapping_easy/mav0/imu0/data.csv",
    header=0,
    names=(
        'timestamp', 
        'accel_x', 'accel_y', 'accel_z', 
        'gyro_x', 'gyro_y', 'gyro_z'
    )
)

df_imu['timestamp'] = pd.to_datetime(df_imu['timestamp'], unit='ns')

df_imu.head()

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
0,1970-01-01 02:24:17.504867890,-0.002508,0.003027,-0.001717,0.213229,-9.653634,1.091423
1,1970-01-01 02:24:17.505867890,-0.001857,0.012511,0.005121,0.194403,-9.662547,1.082578
2,1970-01-01 02:24:17.506863190,-0.001550,-0.016782,-0.000349,0.203321,-9.672472,1.086496
3,1970-01-01 02:24:17.507858390,-0.003004,0.006044,-0.003201,0.218183,-9.701249,1.101222
4,1970-01-01 02:24:17.508848790,-0.000757,-0.000356,-0.002875,0.227100,-9.682409,1.106151


In [6]:
df_image_cam0 = pd.read_csv(
    "D:/thesis_code/datasets/monado_slam/MOO07/MOO07_mapping_easy/mav0/cam0/data.csv",
    header=0,
    names=(
        'timestamp', 
        'filename'
    )
)

df_image_cam0['timestamp'] = pd.to_datetime(df_image_cam0['timestamp'], unit='ns')
df_image_cam0 = df_image_cam0.sort_values(by='timestamp')
df_image_cam0['filename'] = df_image_cam0['filename'].str.replace(".png", "", regex=False)

df_image_cam0.head()

,timestamp,filename
0,1970-01-01 02:24:17.516925124,8657519072990
1,1970-01-01 02:24:17.550229924,8657552377790
2,1970-01-01 02:24:17.583504024,8657585651890
3,1970-01-01 02:24:17.616793524,8657618941390
4,1970-01-01 02:24:17.650083124,8657652230990


In [9]:
def average_last_n_imu_data(imu_df, image_df, N=1_000):
    averaged_imu_data = []

    for _, image_row in image_df.iterrows():
        image_timestamp = image_row["timestamp"]
        filename = image_row["filename"]
        
        # Last N IMU rows before current image timestamp
        # len = 13536
        mask = imu_df["timestamp"] <= image_timestamp 
        imu_before = imu_df[mask].tail(N)

        # Calculate the time span covered by these N rows
        if len(imu_before) > 1:
            time_span_seconds = imu_before["timestamp"].iloc[-1] - imu_before["timestamp"].iloc[0]
        else:
            time_span_seconds = 0  # If there are fewer than 2 rows, span is effectively 0
        
        averaged_imu_data.append({
            "timestamp": image_timestamp,
            "filename": filename,
            
            "avg_accel_x": imu_before["accel_x"].mean(),
            "avg_accel_y": imu_before["accel_y"].mean(),
            "avg_accel_z": imu_before["accel_z"].mean(),
            
            "avg_gyro_x": imu_before["gyro_x"].mean(),
            "avg_gyro_y": imu_before["gyro_y"].mean(),
            "avg_gyro_z": imu_before["gyro_z"].mean(),
            
            "time_span_seconds": time_span_seconds
        })

    return pd.DataFrame(averaged_imu_data)

In [12]:
df = average_last_n_imu_data(df_imu, df_image_cam0, N=10_000)
df.head()

,timestamp,filename,avg_accel_x,avg_accel_y,avg_accel_z,avg_gyro_x,avg_gyro_y,avg_gyro_z,time_span_seconds
0,1970-01-01 02:24:17.516925124,8657519072990,-0.001125,-0.000626,-0.000887,0.224432,-9.666918,1.091622,0 days 00:00:00.011942700
1,1970-01-01 02:24:17.550229924,8657552377790,-0.000754,-0.000406,-0.000090,0.228457,-9.657548,1.090883,0 days 00:00:00.044789800
2,1970-01-01 02:24:17.583504024,8657585651890,-0.000709,-0.000673,0.000151,0.224400,-9.657014,1.091822,0 days 00:00:00.078627500
3,1970-01-01 02:24:17.616793524,8657618941390,-0.000725,-0.000671,0.000106,0.225548,-9.657874,1.093380,0 days 00:00:00.111465100
4,1970-01-01 02:24:17.650083124,8657652230990,-0.000714,-0.000604,0.000092,0.224799,-9.657060,1.092375,0 days 00:00:00.144312200


In [14]:
def average_last_t_seconds_imu(imu_df, image_df, T_seconds=2):
    averaged_imu_data = []

    for _, image_row in image_df.iterrows():
        image_timestamp = image_row["timestamp"]
        filename = image_row["filename"]

        # Start time is T seconds before the current image timestamp
        start_time = image_timestamp - timedelta(seconds=T_seconds)

        # Get IMU rows within [start_time, image_timestamp]
        imu_before = imu_df[(imu_df["timestamp"] > start_time) & (imu_df["timestamp"] <= image_timestamp)]

        # Calculate the time span covered by these rows
        time_span_seconds = 0
        if len(imu_before) > 1:
            time_span_seconds = (imu_before["timestamp"].iloc[-1] - imu_before["timestamp"].iloc[0]).total_seconds()

        averaged_imu_data.append({
            "timestamp": image_timestamp,
            "filename": filename,

            "avg_accel_x": imu_before["accel_x"].mean(),
            "avg_accel_y": imu_before["accel_y"].mean(),
            "avg_accel_z": imu_before["accel_z"].mean(),

            "avg_gyro_x": imu_before["gyro_x"].mean(),
            "avg_gyro_y": imu_before["gyro_y"].mean(),
            "avg_gyro_z": imu_before["gyro_z"].mean(),

            "time_span_seconds": time_span_seconds
        })

    df = pd.DataFrame(averaged_imu_data)
    return df

In [16]:
df = average_last_t_seconds_imu(df_imu, df_image_cam0, T_seconds=2)
df.tail()

,timestamp,filename,avg_accel_x,avg_accel_y,avg_accel_z,avg_gyro_x,avg_gyro_y,avg_gyro_z,time_span_seconds
1232,1970-01-01 02:24:58.529442824,8698531590690,0.004397,0.001894,-0.000094,0.205163,-9.663521,1.052548,1.999407
1233,1970-01-01 02:24:58.562726224,8698564874090,0.004705,0.002045,0.000186,0.205406,-9.661393,1.055156,1.998407
1234,1970-01-01 02:24:58.596015024,8698598162890,0.004660,0.002034,0.000126,0.204902,-9.660982,1.056588,1.999397
1235,1970-01-01 02:24:58.629303424,8698631451290,0.004622,0.001930,0.000020,0.204453,-9.661824,1.058936,1.998407
1236,1970-01-01 02:24:58.662593824,8698664741690,0.003958,0.001612,0.000237,0.204559,-9.659111,1.061471,1.999402
